In [93]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt

Variables:

In [94]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

Valor real de la integral:

In [95]:
Exacto = (np.pi**4)/15
Exacto

6.493939402266828

In [96]:
def GetLaguerre(n,x):
    
    if n==0:
        LN=1
    elif n==1:
        LN=(1-x)
    else:    
        LN = ((2*n-1-x)*GetLaguerre(n-1,x)-(n-1)*GetLaguerre(n-2,x))/n
    return LN


In [97]:
def GetDLeguerre(n,x):
    Pn = GetLaguerre(n,x)
    return sym.diff(Pn,x,1)

In [98]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [99]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [100]:
def GetAllRootsGLeg(n):

    xn = np.linspace(0,n+(n-1)*np.sqrt(n),100)
    
    Leguerre = []
    DLeguerre = []
    
    for i in range(n+1):
        Leguerre.append(GetLaguerre(i,x))
        DLeguerre.append(GetDLeguerre(i,x))
    
    poly = sym.lambdify([x],Leguerre[n],'numpy')
    Dpoly = sym.lambdify([x],DLeguerre[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [101]:
def GetWeightsGLag(n):
    Roots = GetAllRootsGLeg(n)
    Dleguerre=[]
    for i in range(n+1):
        Dleguerre.append(GetDLeguerre(i,x))
    
    Dpoly = sym.lambdify([x],Dleguerre[n],'numpy')
    Weights = Roots/(((n+1)**2)*(GetLaguerre(n+1,Roots)**2))
    
    return Weights

In [102]:
def todo (n):
    raices = GetAllRootsGLeg(n)
    pesos = GetWeightsGLag(n)
    funcion = lambda x: ((x**3)*(np.e**x))/((np.e**x)-1)
    I = 0
    for i in range(n):
        I += pesos[i]*funcion(raices[i])
    return I


In [103]:
i=todo(3)
i

6.4811301714277665

Error y grafica:

In [104]:
d=[]
j=[]
for i in range (2,11):
    d.append(i)
    f=todo(i)
    f=f/Exacto
    j.append(f)

plot=plt.scatter(d,j)
plot
